# Dependencies

In [ ]:
print('running previous notebook from mriqc_exploration:')
%run '/Users/rodgersleejg/data/hpc/NNDSP/anal/mriqc_files/analysis_notebooks/merge_manual_qc_files.ipynb'

# Setup 

In [ ]:
from pathlib import Path
import pandas as pd
import numpy as np
import subprocess
from datetime import date
import re
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('max_colwidth',500)
%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt
from importlib import reload


In [ ]:
## EDIT THESE VARIABLES
analysis_version = "2017_10_19"
project_dir = Path('/Users/rodgersleejg/data/hpc/NNDSP') # needs to be pathlib.Path object

In [ ]:
bids_dir = project_dir.joinpath('bids_2017_07_14')

# conf_script = mriqc_dir.joinpath('conf' + analysis_version + '.sh')

mriqc_dir  = project_dir.joinpath('anal/mriqc_files/other_files')
if not mriqc_dir.exists():
    mriqc_dir.mkdir()
output_folder =  project_dir / 'derivatives' / 'mriqc'
if not output_folder.exists():
    output_folder.mkdir()
classifier_output =  output_folder.joinpath('classifier')
if not classifier_output.exists():
    classifier_output.mkdir()
base_work_dir = output_folder.joinpath('work')
if not base_work_dir.exists():
    base_work_dir.mkdir()
log_dir = mriqc_dir.joinpath('swarm_output_' +  analysis_version)
if not log_dir.exists():
    log_dir.mkdir()
manual_qc = output_folder.joinpath('manual_qc_round_2.tsv')
# /Users/rodgersleejg/data/hpc/NNDSP/derivatives/mriqc/manual_qc_round_2.tsv
# swarm_path = mriqc_dir.joinpath('mriqc_' + analysis_version + '.cmd')


mriqc_with_predictions = Path('derivatives/mriqc/with_mriqc_predictions.csv')
plottable_data = Path('derivatives/mriqc/classifer_plot_data.pklz')
plottable_data_euler = Path('derivatives/mriqc/classifer_plot_data_euler.pklz')
qc_data = Path('anal/mriqc_files/other_files/qc_pickle_for_v2_exploration.pklz')

In [ ]:
%pwd
%cd {project_dir}
%pwd

In [ ]:
import anal.python_modules.inner_merge_and_report as pd_custom

# Using MRIQC on their data (DS030)

This version of mriqc was downloaded more recently than the paper publication. Some of the IQMs  have changed but it is roughly the same.

With ghost included in test set:

With ghost excluded in test set:

# For NNDSP data get mriqc classifications:

In [ ]:
feature_file = Path('derivatives/mriqc/x_nndsp.csv')
if not feature_file.exists():
    df_example = pd.read_csv('/home/rodgersleejg/x_ds030.csv')
    cols = df_example.columns
    cols = [cols[0],'run_id',*cols[1:]]
    t1 = pd.read_csv('derivatives/mriqc/T1w.csv')
    t1.loc[:,cols].to_csv(feature_file,index = False)

else:
    t1 = pd.read_csv(feature_file)

# t1.loc[:,cols].loc[t1.subject_id == 1000,:]

In [ ]:
classifier_output

In [ ]:
ls {classifier_output}

In [ ]:
reload(pd_custom)
if mriqc_with_predictions.exists():
    df_mriqc = pd.read_csv('derivatives/mriqc/with_mriqc_predictions.csv')
else:
    df_pred = pd.read_csv(classifier_output.joinpath('mclf_run-20171027-140418_data-unseen_pred.csv'))
    df_mriqc = pd_custom.merge_and_report(df_pred,t1, on = ['subject_id','run_id'])
    df_mriqc.to_csv(mriqc_with_predictions,index = False)

df_mriqc['run'] = df_mriqc.run_id.apply(lambda x: 'run-' + '{n:03d}'.format(n = x))
df_mriqc.head(1)

# Merge with NNDSP manual qc

## Filter out subjects with multiple scans

Some MASKIDs have multiple runs and it is unclear which scan the manual qc is referring to. So first I will get all the scans that don't have duplicate runs:

In [ ]:
bids_dir

In [ ]:
df_scans = pd.DataFrame(columns=['scan_path'],data=[p.as_posix() for p in bids_dir.glob('**/anat/*T1w.nii.gz')])
# df_scans = df_scans.assign(subject = lambda df: df.scan_path.str.extract('(sub-\d{2,4})', expand=True))
df_scans = pd.concat(
    [df_scans,
     df_scans.scan_path.
     str.extract(
         '.*(?P<subject>sub-\d{2,4})/.*(?P<run>run-[a-zA-Z0-9]{1,8})_.*',
         expand=True)],
    axis = 1)
df_scans['MASKID']  = df_scans.subject.str.replace('sub-','')

print(len(df_scans))
df_scans.head(2)

In [ ]:
(df_scans.scan_path.nunique(), df_scans.subject.nunique(), df_scans.run.nunique())

In [ ]:
df_scans_filtered = df_scans


### Load manual qc data

In [ ]:
df_manual = pd.read_csv(manual_qc,sep ='\t')

In [ ]:
manual_qc

In [ ]:
df_manual

In [ ]:
difficult_col1 = [c for c in df_manual.columns if c.find('Freesurfer_int')==0 and c.find('discrep')>0][0]
difficult_col2 = 'Freesurfer_avg_int_rating '
df_manual = (df_manual.
             rename(columns = 
                    {
                        difficult_col1 : 'Freesurfer_int_discrepency',
                        difficult_col2 : 'Freesurfer_avg_int_rating'
                    }
                    
                   )
            )
df_manual['MASKID'] = df_manual.MASKID.apply(lambda x: '{n:04d}'.format(n = x))
df_manual['run'] = df_manual.run.apply(lambda x: 'run-' + '{n:03d}'.format(n = x))
print(len(df_manual))
# df_manual.loc[df_manual.duplicated(['MASKID'],keep = False),:].sort_values('MASKID')
# df_manual.loc[df_manual.duplicated(['MASKID','run'],keep = False),:].sort_values('MASKID')

### Apply filter to manual qc and merge with scan paths

In [ ]:
print('Num ratings: ', len(df_manual), ' Unique mask_ids: ', df_manual.MASKID.nunique())


In [ ]:
len(df_scans_filtered)

In [ ]:
reload(pd_custom)
# df_qc = pd_custom.merge_and_report(df_manual, df_scans_filtered, on = ['MASKID'])
df_qc = df_manual.merge(df_scans_filtered, on = ['MASKID','run'], how = 'outer',indicator = True)
# there are tarred dicoms for the below files. they must have not been extracted by heudiconv.

#### Some manual ratings didn't have scans extracted for them:


In [ ]:
# df_qc.query("MASKID == '1360'")
df_qc.query("_merge != 'both'")

In [ ]:
df_qc = df_qc.query("_merge == 'both'").drop( '_merge', axis = 0, errors ='ignore')

In [ ]:
print(len(df_qc))

### Merge with mriqc values

In [ ]:
# df_mriqc = df_mriqc.query('run_id == 1')
df_mriqc['MASKID'] = df_mriqc.subject_id.apply(lambda x: '{n:04d}'.format(n = x))

df_qc_full = df_qc.copy().merge(df_mriqc,how = 'inner',on = ['MASKID','run'])
print(len(df_mriqc), len(df_qc_full))
df_qc_full.head()

In [ ]:
print(len(df_mriqc), len(df_qc_full),' File: ', qc_data)
df_qc_full.to_pickle(qc_data)